## Making the datawarehouse 

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import pandasql as ps
conn = sqlite3.connect('CourseDataWarehouse.db')
c = conn.cursor()

In [2]:
% load_ext sql

In [3]:
%%sql

sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [4]:
%%sql

DROP TABLE IF EXISTS LOCATION_W;

CREATE TABLE LOCATION_W(
    LID INTEGER PRIMARY KEY,
    BUILDING TEXT,
    ROOM TEXT
);

DROP TABLE IF EXISTS FACULTY_W;

CREATE TABLE FACULTY_W(
    FID INTEGER PRIMARY KEY,
    LNAME TEXT,
    FNAME TEXT
);

DROP TABLE IF EXISTS COURSE_W;

CREATE TABLE COURSE_W(
    CID INTEGER PRIMARY KEY,
    CRN TEXT,
    CREDITS TEXT,
    PREREQS TEXT,
    COREQS TEXT,
    ATTRIBUTES TEXT,
    DESCRIPTION TEXT,
    PROGRAM_NAME TEXT,
    PROGRAM_CODE TEXT
);

DROP TABLE IF EXISTS TERM_W;

CREATE TABLE TERM_W(
    TERMID INTEGER PRIMARY KEY,
    SEMESTER TEXT,
    YEAR TEXT,
    TERM TEXT
);


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## populating the dimension tables

In [5]:
%%sql

--popluting the location dimension table--
INSERT INTO LOCATION_W(BUILDING, ROOM)
SELECT DISTINCT BUILDING, ROOM 
FROM Location;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [6]:
%%sql

--populating the faculty_w dimension table--
INSERT INTO FACULTY_W(LNAME, FNAME)
SELECT DISTINCT LNAME, FNAME
FROM FACULTY;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [7]:
%%sql

--populating course_w dimension table
INSERT INTO COURSE_W(CRN,CREDITS,PREREQS, COREQS,ATTRIBUTES,DESCRIPTION,PROGRAM_NAME,PROGRAM_CODE)
SELECT DISTINCT CRN, CREDITS,PREREQS, COREQS,ATTRIBUTES,DESCRIPTION,PROGRAMNAME,PROGRAMCODE
FROM COURSE
    INNER JOIN COURSE_OFFERING ON (COURSE.CID = COURSE_OFFERING.CID)
    JOIN PROGRAMS USING (PID)
;
    
    


 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [8]:
%%sql

--populating the term_w table--
INSERT INTO TERM_W(SEMESTER, YEAR, TERM)
SELECT DISTINCT SEMESTER, YEAR, TERM
FROM COURSE_OFFERING;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

## Creating the fact table.

In [9]:
%%sql

DROP TABLE IF EXISTS COURSE_OFFERING_W;

DROP TABLE IF EXISTS CLASS_FACTS_W;

CREATE TABLE CLASS_FACTS_W(
    COID INTEGER PRIMARY KEY,
    CAP INTEGER,
    ACT INTEGER, 
    REM INTEGER,
    COUNT_OF_FACULTY_PER_PROGRAM INTEGER,
    COUNT_OF_COURSES INTEGER,
    MAX_ACT INTERGER,
    AVG_ACT INTERGER,
    COUNT_CLASSES_PER_BLDG INTEGER,
    TERMID INTEGER,
    CID INTEGER,
    LID INTEGER,
    FID INTEGER,
    FOREIGN KEY (TERMID) REFERENCES TERM_W (TERMID),
    FOREIGN KEY (CID) REFERENCES COURSE_W (CID),
    FOREIGN KEY (LID) REFERENCES LOCATION_W (LID),
    FOREIGN KEY (FID) REFERENCES FACULTY_W (FID)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.


[]

In [10]:
%%sql

--populating the CLASS_FACTS_W table
INSERT INTO CLASS_FACTS_W (CAP, ACT, REM, TERMID, CID, LID, FID)
SELECT DISTINCT CAP, ACT, REM, TERMID, CID,LID, FID
FROM COURSE_OFFERING
    JOIN FACULTY_W USING (FID)
    JOIN CLASS_MEETINGS USING (COID)
    JOIN LOCATION USING (LID)
    JOIN TERM_W USING (SEMESTER)
    JOIN COURSE_W USING (CID)
;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

## Manipulating the Fact Table to fill in numeric data.

### Use pandas to write as dataframe and export as database table.

### Fill in 'MAX_ACT' & 'AVG_ACT

In [11]:
table = '''SELECT * FROM CLASS_FACTS_W'''
df = pd.read_sql(table, conn)
df.head()


,COID,CAP,ACT,REM,COUNT_OF_FACULTY_PER_PROGRAM,COUNT_OF_COURSES,MAX_ACT,AVG_ACT,COUNT_CLASSES_PER_BLDG,TERMID,CID,LID,FID
0,1,27,27,0,None,None,None,None,None,1,1369,1,125
1,2,27,27,0,None,None,None,None,None,2,1369,1,125
2,3,27,27,0,None,None,None,None,None,3,1369,1,125
3,4,27,27,0,None,None,None,None,None,4,1369,1,125
4,5,27,27,0,None,None,None,None,None,5,1369,1,125


In [12]:
q1 = '''SELECT AVG(ACT) as avg_act, CID FROM df GROUP BY CID'''
df2=pd.DataFrame(ps.sqldf(q1, locals()))

q2 = '''SELECT MAX(CAP) as max_act, CID FROM df GROUP BY CID'''
df3 = pd.DataFrame(ps.sqldf(q2, locals()))
df2.head()



,avg_act,CID
0,1.00000,1
1,1.00000,2
2,23.74359,3
3,10.00000,4
4,12.00000,7


In [13]:
df3.head()

,max_act,CID
0,0,1
1,5,2
2,25,3
3,25,4
4,25,7


In [14]:
# merge the little dataframes onto the big one
df=df.merge(df2, on='CID', how='left')
df = df.merge(df3, on='CID', how = 'left')

#set the empty columns to the newly added colums
df['MAX_ACT'] = df['max_act']
df['AVG_ACT'] = df['avg_act']

#drop the intermediate columns
#yes, this could have been done more elegantly...
df = df.drop(['avg_act', 'max_act'], axis = 1)
df.head(5)

,COID,CAP,ACT,REM,COUNT_OF_FACULTY_PER_PROGRAM,COUNT_OF_COURSES,MAX_ACT,AVG_ACT,COUNT_CLASSES_PER_BLDG,TERMID,CID,LID,FID
0,1,27,27,0,None,None,27,23.317919,None,1,1369,1,125
1,2,27,27,0,None,None,27,23.317919,None,2,1369,1,125
2,3,27,27,0,None,None,27,23.317919,None,3,1369,1,125
3,4,27,27,0,None,None,27,23.317919,None,4,1369,1,125
4,5,27,27,0,None,None,27,23.317919,None,5,1369,1,125


### Fill in number of CID's for program code.

These values will be incorrect due to our bug in the previous steps of or COURSE_W table.


In [15]:

df4 = pd.read_sql('''
SELECT  COUNT( DISTINCT CID), PROGRAM_CODE, CID
FROM CLASS_FACTS_W
    JOIN COURSE_W USING (CID)
GROUP BY PROGRAM_CODE;''', con = conn)

df = df.merge(df4, how = 'inner', on='CID')
df['COUNT_OF_COURSES'] = df['COUNT( DISTINCT CID)']
df = df.drop(['COUNT( DISTINCT CID)', 'PROGRAM_CODE'], axis = 1)
df.head()

,COID,CAP,ACT,REM,COUNT_OF_FACULTY_PER_PROGRAM,COUNT_OF_COURSES,MAX_ACT,AVG_ACT,COUNT_CLASSES_PER_BLDG,TERMID,CID,LID,FID
0,1,27,27,0,None,19,27,23.317919,None,1,1369,1,125
1,2,27,27,0,None,19,27,23.317919,None,2,1369,1,125
2,3,27,27,0,None,19,27,23.317919,None,3,1369,1,125
3,4,27,27,0,None,19,27,23.317919,None,4,1369,1,125
4,5,27,27,0,None,19,27,23.317919,None,5,1369,1,125


In [16]:
df.sample(5)

,COID,CAP,ACT,REM,COUNT_OF_FACULTY_PER_PROGRAM,COUNT_OF_COURSES,MAX_ACT,AVG_ACT,COUNT_CLASSES_PER_BLDG,TERMID,CID,LID,FID
783,162,28,27,1,None,54,29,28.100000,None,2,739,36,622
583,24051,19,17,2,None,37,25,18.301038,None,6,1288,80,171
654,27183,25,25,0,None,61,30,26.857143,None,8,526,171,467
669,58,15,14,1,None,72,15,14.000000,None,3,1361,14,164
1869,3595,25,21,4,None,12,25,22.500000,None,5,2855,2,877


### Fill in the count of faculty per program code.

These values are also incorrect.

In [17]:
##get the count of faculty by program code

df5 = pd.read_sql('''
SELECT COUNT(DISTINCT FID), PROGRAM_CODE, CID
FROM CLASS_FACTS_W
    JOIN COURSE_W USING (CID)
    GROUP BY PROGRAM_CODE''',
                  con = conn)

df5.head(5)

,COUNT(DISTINCT FID),PROGRAM_CODE,CID
0,90,AC,1448
1,35,AE,66
2,25,AH,1520
3,3,AN,2980
4,1,AR,2964


In [18]:
df = df.merge(df5, how = 'inner', on='CID')
df['COUNT_OF_FACULTY_PER_PROGRAM'] = df['COUNT(DISTINCT FID)']
df = df.drop(['COUNT(DISTINCT FID)', 'PROGRAM_CODE'], axis = 1)
df.head()

,COID,CAP,ACT,REM,COUNT_OF_FACULTY_PER_PROGRAM,COUNT_OF_COURSES,MAX_ACT,AVG_ACT,COUNT_CLASSES_PER_BLDG,TERMID,CID,LID,FID
0,1,27,27,0,25,19,27,23.317919,None,1,1369,1,125
1,2,27,27,0,25,19,27,23.317919,None,2,1369,1,125
2,3,27,27,0,25,19,27,23.317919,None,3,1369,1,125
3,4,27,27,0,25,19,27,23.317919,None,4,1369,1,125
4,5,27,27,0,25,19,27,23.317919,None,5,1369,1,125


In [19]:
df6 = pd.read_sql('''SELECT * FROM CLASS_FACTS_W JOIN LOCATION USING (LID)''', conn)


### Fill in the number of classes per building. 

In [20]:
q2 ='''SELECT COUNT(DISTINCT CID) AS COUNT_OF_CLASS_PER_BUILDING, LID FROM df6 GROUP BY LID'''
df10 = pd.DataFrame(ps.sqldf(q2, locals()))
df10.head()

,COUNT_OF_CLASS_PER_BUILDING,LID
0,70,1
1,67,2
2,44,3
3,59,4
4,47,5


In [21]:
df = df.merge(df10, on='LID', how='left')
df['COUNT_CLASSES_PER_BLDG'] = df['COUNT_OF_CLASS_PER_BUILDING']
df.drop('COUNT_OF_CLASS_PER_BUILDING', axis = 1)
df.head()

,COID,CAP,ACT,REM,COUNT_OF_FACULTY_PER_PROGRAM,COUNT_OF_COURSES,MAX_ACT,AVG_ACT,COUNT_CLASSES_PER_BLDG,TERMID,CID,LID,FID,COUNT_OF_CLASS_PER_BUILDING
0,1,27,27,0,25,19,27,23.317919,70,1,1369,1,125,70
1,2,27,27,0,25,19,27,23.317919,70,2,1369,1,125,70
2,3,27,27,0,25,19,27,23.317919,70,3,1369,1,125,70
3,4,27,27,0,25,19,27,23.317919,70,4,1369,1,125,70
4,5,27,27,0,25,19,27,23.317919,70,5,1369,1,125,70


In [22]:
df.to_sql('CLASS_FACTS_W', conn, if_exists = 'replace')

## Drop unnecessary tables

(Keep commented out for now)

In [23]:
%%sql

--DROP TABLE COURSE;
--DROP TABLE PROGRAMS;
--DROP TABLE COURSE_OFFERING;
--DROP TABLE CLASS_MEETINGS;
--DROP TABLE FACULTY;
--DROP TABLE LOCATION;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.


[]

In [24]:
%%sql

--VACUUM

 * sqlite:///CourseDataWarehouse.db
0 rows affected.


[]